# 🚀 Operational Flow:

**Executes the entire flow with a notebook interface:**
- Initiate DB
- Load CSV to DB
- Load (and validate) TAK repository
- Apply TAKs on DB
- View output

In [ ]:
# !pip install -r requirements-py37.txt
# !pip install -e .

In [1]:
from pathlib import Path
from backend.dataaccess import DataAccess
from core.mediator import Mediator
import pandas as pd

In [2]:
# Paths
KB_PATH = Path("core/knowledge-base")
DB_PATH = Path("backend/data/mediator.db")
CSV_PATH = Path("backend/data/synthetic_input_data.csv")

In [3]:
# 1. Connect to existing DB
da = DataAccess(db_path=str(DB_PATH))

# # 2. Or auto-create (and optionally drop existing)
# da = DataAccess(db_path=str(DB_PATH), auto_create=True)

# Check stats
stats = da.get_table_stats()
for table, info in stats.items():
    print(f"{table}: {info['rows']} rows, {info['n_patients']} patients")

InputPatientData: 1778 rows, 12 patients
OutputPatientData: 162 rows, 2 patients
PatientQAScores: 0 rows, 0 patients


In [4]:
query = """
SELECT PatientId, ConceptName, StartDateTime, EndDateTime, Value
FROM InputPatientData
WHERE PatientId = 1000
"""
df_results = pd.read_sql_query(query, da.conn)
df_results

,PatientId,ConceptName,StartDateTime,EndDateTime,Value
0,1000,ADMISSION,2025-01-21 09:00:00,2025-01-21 09:00:01,True
1,1000,DIABETES_DIAGNOSIS,2025-01-21 09:00:00,2025-01-21 09:00:01,True
2,1000,GLUCOSE_MEASURE,2025-01-21 12:00:00,2025-01-21 12:00:01,172.4
3,1000,MEAL,2025-01-21 13:00:00,2025-01-21 13:00:01,Lunch
4,1000,BOLUS_DOSAGE,2025-01-21 13:07:00,2025-01-21 13:07:01,14.6
...,...,...,...,...,...
58,1000,GLUCOSE_MEASURE,2025-01-24 12:00:00,2025-01-24 12:00:01,135.4
59,1000,BOLUS_DOSAGE,2025-01-24 12:51:00,2025-01-24 12:51:01,11.2
60,1000,BOLUS_ROUTE,2025-01-24 12:51:00,2025-01-24 12:51:01,IntraVenous
61,1000,MEAL,2025-01-24 13:00:00,2025-01-24 13:00:01,Lunch


In [ ]:
# Load CSV into InputPatientData
total_rows = da.load_csv_to_input(
    csv_path=str(CSV_PATH),
    if_exists='append',           # 'append' or 'replace'
    clear_output_and_qa=False,    # Set True to clear outputs
    yes=True                      # Auto-confirm
)
print(f"Loaded {total_rows} rows")

In [5]:
# Initialize mediator
mediator = Mediator(knowledge_base_path=KB_PATH, data_access=da)

# Build TAK repository
repo = mediator.build_repository()

print(f"✅ Loaded {len(repo.taks)} TAKs:")
print(f"  - Raw Concepts: {len(mediator.raw_concepts)}")
print(f"  - Events:       {len(mediator.events)}")
print(f"  - States:       {len(mediator.states)}")
print(f"  - Trends:       {len(mediator.trends)}")
print(f"  - Contexts:     {len(mediator.contexts)}")
print(f"  - Patterns:     {len(mediator.patterns)}")

# List all TAK names
print("\nTAK Names:")
for tak_name in sorted(repo.taks.keys()):
    print(f"  - {tak_name}")


PHASE 1: Building TAK Repository


Loading TAKs:   0%|          | 0/37 [00:00<?, ?file/s]

Loading TAKs: 100%|██████████| 37/37 [00:00<00:00, 279.73file/s, Patterns: INSULIN_ON_HIGH_GLUCOSE_PATTERN]     


[Validation] Running business-logic checks on TAK repository...

✅ TAK Repository Built Successfully
  Raw Concepts: 15
  Events:       4
  States:       5
  Trends:       2
  Contexts:     6
  Patterns:     5
  TOTAL TAKs:   37

✅ Loaded 37 TAKs:
  - Raw Concepts: 15
  - Events:       4
  - States:       5
  - Trends:       2
  - Contexts:     6
  - Patterns:     5

TAK Names:
  - ADMISSION
  - ADMISSION_EVENT
  - ANTIDIABETIC_DRUGS_IV_BITZUA
  - ANTIDIABETIC_DRUGS_IV_BITZUA_CONTEXT
  - ANTIDIABETIC_DRUGS_IV_BITZUA_STATE
  - BASAL_BITZUA
  - BASAL_BITZUA_CONTEXT
  - BASAL_BITZUA_STATE
  - BMI_MEASURE
  - BMI_MEASURE_ON_ADMISSION
  - BOLUS_BITZUA
  - BOLUS_BITZUA_CONTEXT
  - BOLUS_BITZUA_STATE
  - CREATININE_MEASURE_ON_ADMISSION
  - CREATININE_SERUM_MEASURE
  - CREATININE_SERUM_MEASURE_STATE
  - CREATININE_SERUM_MEASURE_TREND
  - DEATH
  - DEATH_EVENT
  - DIABETES_DIAGNOSIS
  - DIABETES_DIAGNOSIS_CONTEXT
  - DISGLYCEMIA_EVENT
  - GLUCOSE_MEASURE
  - GLUCOSE_MEASURE_ON_ADMISSION_PATTER

In [ ]:
# Process specific patients (Jupyter-compatible)
patient_ids = [1000, 1001, 1002]
patient_stats = await mediator.run_async(
    max_concurrent=4,
    patient_subset=patient_ids
)

# Print results
for pid, stats in patient_stats.items():
    if "error" in stats:
        print(f"❌ Patient {pid}: {stats['error']}")
    else:
        total = sum(v for k, v in stats.items() if isinstance(v, int))
        print(f"✅ Patient {pid}: {total} output rows")


PHASE 1: Building TAK Repository


Loading TAKs: 100%|██████████| 37/37 [00:00<00:00, 231.76file/s, Patterns: INSULIN_ON_HIGH_GLUCOSE_PATTERN]     



[Validation] Running business-logic checks on TAK repository...

✅ TAK Repository Built Successfully
  Raw Concepts: 60
  Events:       16
  States:       20
  Trends:       8
  Contexts:     24
  Patterns:     20
  TOTAL TAKs:   37


PHASE 2: Processing 12 Patients (max_concurrent=4)



Processing patients: 100%|██████████| 12/12 [00:05<00:00,  2.40patient/s]


✅ Patient Processing Complete
  Patients processed: 12
  Total rows written: 2742
  Errors:             0

✅ Patient 1000: 83 output rows
✅ Patient 1001: 66 output rows
✅ Patient 1002: 345 output rows
✅ Patient 1003: 366 output rows
✅ Patient 1004: 122 output rows
✅ Patient 1005: 303 output rows
✅ Patient 1006: 348 output rows
✅ Patient 1007: 229 output rows
✅ Patient 1008: 366 output rows
✅ Patient 1009: 150 output rows
✅ Patient 1010: 287 output rows
✅ Patient 1011: 77 output rows


In [15]:
# Query OutputPatientData
query = """
SELECT PatientId, ConceptName, StartDateTime, EndDateTime, Value
FROM OutputPatientData
"""
df_results = pd.read_sql_query(query, da.conn)
df_results

,PatientId,ConceptName,StartDateTime,EndDateTime,Value
0,1002,DISGLYCEMIA_EVENT,2025-01-06 20:00:00,2025-01-06 20:00:01,Hyperglycemia
1,1000,DISGLYCEMIA_EVENT,2025-01-22 16:00:00,2025-01-22 16:00:01,Hypoglycemia
2,1000,DISGLYCEMIA_EVENT,2025-01-22 20:00:00,2025-01-22 20:00:01,Hypoglycemia
3,1000,DISGLYCEMIA_EVENT,2025-01-23 08:00:00,2025-01-23 08:00:01,Hypoglycemia
4,1000,DISGLYCEMIA_EVENT,2025-01-24 06:15:00,2025-01-24 06:15:01,Hypoglycemia
...,...,...,...,...,...
1022,1005,MEAL_CONTEXT,2025-01-23 06:00:00,2025-01-23 10:00:01,Breakfast
1023,1005,MEAL_CONTEXT,2025-01-23 11:00:00,2025-01-23 15:00:01,Lunch
1024,1005,MEAL_CONTEXT,2025-01-23 17:00:00,2025-01-23 21:00:01,Dinner
1025,1005,MEAL_CONTEXT,2025-01-24 06:00:00,2025-01-24 10:00:01,Breakfast


# 🔍 TAK Debugging Toolkit

Interactive debugging for a single patient's abstraction flow.

**Features:**
- Load minimal data (only concepts relevant to selected TAKs)
- Use isolated debug DB (no production contamination)
- Full TAK execution trace with cache inspection
- Compare input → intermediate → output at each layer

In [9]:
# ============================================================
# Configuration: Select Patient & TAKs to Debug
# ============================================================

DEBUG_PATIENT_ID = 1000

# Select TAKs to trace (in dependency order)
DEBUG_TAKS = [
    # Raw concepts
    "DIABETES_DIAGNOSIS",
    "ADMISSION",
    "GLUCOSE_MEASURE",
    "BASAL_BITZUA",
    
    # Events
    "ADMISSION_EVENT",
    
    # Contexts
    "DIABETES_DIAGNOSIS_CONTEXT",
    
    # Patterns
    # "INSULIN_ON_ADMISSION_PATTERN",
]

# Debug DB path (separate from production)
DEBUG_DB_PATH = Path("backend/data/debug_mediator.db")

In [10]:
# ============================================================
# Step 1: Extract Minimal Input Data for Debug Patient
# ============================================================

# Query production DB for this patient's input data
query = f"""
SELECT DISTINCT PatientId, ConceptName, StartDateTime, EndDateTime, Value
FROM InputPatientData
WHERE PatientId = {DEBUG_PATIENT_ID}
ORDER BY StartDateTime, ConceptName
"""

df_patient_input = pd.read_sql_query(query, da.conn)

print(f"📥 Input data for patient {DEBUG_PATIENT_ID}:")
print(f"   Total rows: {len(df_patient_input)}")
print(f"\n📊 Unique concepts in input:")
for concept in sorted(df_patient_input['ConceptName'].unique()):
    count = (df_patient_input['ConceptName'] == concept).sum()
    print(f"   - {concept}: {count} rows")

# Show sample
display(df_patient_input.head(20))

📥 Input data for patient 1000:
   Total rows: 63

📊 Unique concepts in input:
   - ADMISSION: 1 rows
   - BASAL_DOSAGE: 3 rows
   - BASAL_ROUTE: 3 rows
   - BMI_MEASURE: 1 rows
   - BOLUS_DOSAGE: 10 rows
   - BOLUS_ROUTE: 10 rows
   - DIABETES_DIAGNOSIS: 1 rows
   - GLUCOSE_MEASURE: 22 rows
   - MEAL: 10 rows
   - RELEASE: 1 rows
   - WEIGHT_MEASURE: 1 rows


,PatientId,ConceptName,StartDateTime,EndDateTime,Value
0,1000,ADMISSION,2025-01-21 09:00:00,2025-01-21 09:00:01,True
1,1000,DIABETES_DIAGNOSIS,2025-01-21 09:00:00,2025-01-21 09:00:01,True
2,1000,GLUCOSE_MEASURE,2025-01-21 12:00:00,2025-01-21 12:00:01,172.4
3,1000,MEAL,2025-01-21 13:00:00,2025-01-21 13:00:01,Lunch
4,1000,BOLUS_DOSAGE,2025-01-21 13:07:00,2025-01-21 13:07:01,14.6
5,1000,BOLUS_ROUTE,2025-01-21 13:07:00,2025-01-21 13:07:01,IntraVenous
6,1000,BMI_MEASURE,2025-01-21 15:00:00,2025-01-21 15:00:01,25.3
7,1000,WEIGHT_MEASURE,2025-01-21 15:00:00,2025-01-21 15:00:01,83.9
8,1000,GLUCOSE_MEASURE,2025-01-21 16:00:00,2025-01-21 16:00:01,216.2
9,1000,MEAL,2025-01-21 19:00:00,2025-01-21 19:00:01,Dinner


In [11]:
# ============================================================
# Step 2: Create Debug DB and Load Minimal Data
# ============================================================

# Remove old debug DB if exists
if DEBUG_DB_PATH.exists():
    DEBUG_DB_PATH.unlink()
    print(f"🗑️  Deleted old debug DB: {DEBUG_DB_PATH}")

# Create new debug DB
debug_da = DataAccess(db_path=str(DEBUG_DB_PATH), auto_create=True)

# Load ONLY concepts relevant to DEBUG_TAKS
# Build list of input concepts needed by selected TAKs
mediator_temp = Mediator(knowledge_base_path=KB_PATH, data_access=debug_da)
repo_temp = mediator_temp.build_repository()

needed_concepts = set()
for tak_name in DEBUG_TAKS:
    tak = repo_temp.get(tak_name)
    if tak:
        if hasattr(tak, 'derived_from'):
            if isinstance(tak.derived_from, list):
                # Event/Context/Pattern
                for df in tak.derived_from:
                    needed_concepts.add(df['name'])
            else:
                # State/Trend
                needed_concepts.add(tak.derived_from)
        
        # Add attributes for raw-concepts
        if hasattr(tak, 'attributes'):
            for attr in tak.attributes:
                needed_concepts.add(attr['name'])

print(f"\n🎯 Concepts needed for selected TAKs:")
print(f"   {sorted(needed_concepts)}")

# Filter input data to only needed concepts
df_filtered = df_patient_input[df_patient_input['ConceptName'].isin(needed_concepts)].copy()

print(f"\n📦 Filtered input data:")
print(f"   Rows: {len(df_patient_input)} → {len(df_filtered)}")
print(f"   Concepts: {df_patient_input['ConceptName'].nunique()} → {df_filtered['ConceptName'].nunique()}")

# Rename columns to match InputPatientData schema
df_filtered = df_filtered.rename(columns={
    'StartDateTime': 'StartTime',
    'EndDateTime': 'EndTime'
})

# Load to debug DB
if not df_filtered.empty:
    df_filtered.to_sql('InputPatientData', debug_da.conn, if_exists='append', index=False)
    print(f"\n✅ Loaded {len(df_filtered)} rows to debug DB")
else:
    print(f"\n⚠️  No data to load (needed concepts not found in input)")

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'backend\\data\\debug_mediator.db'

In [ ]:
# ============================================================
# Step 3: Run Mediator with TAK Cache Tracing
# ============================================================

# Initialize debug mediator
debug_mediator = Mediator(knowledge_base_path=KB_PATH, data_access=debug_da)
debug_repo = debug_mediator.build_repository()

# Enable detailed logging
import logging
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

# Run processing for debug patient
print(f"\n🚀 Processing patient {DEBUG_PATIENT_ID} with cache tracing...\n")
print("="*80)

# Process patient synchronously (for easier debugging)
stats = debug_mediator._process_patient_sync(DEBUG_PATIENT_ID)

print("\n" + "="*80)
print(f"\n✅ Processing complete!")
print(f"\n📊 Output stats:")
for tak_name, count in stats.items():
    if isinstance(count, int) and count > 0:
        print(f"   - {tak_name}: {count} rows")

In [ ]:
# ============================================================
# Step 4: Inspect TAK Cache (Intermediate Outputs)
# ============================================================

# Query output for each selected TAK
print(f"\n🔍 TAK-by-TAK Output Inspection:\n")

for tak_name in DEBUG_TAKS:
    tak = debug_repo.get(tak_name)
    if not tak:
        print(f"❌ {tak_name}: TAK not found in repository")
        continue
    
    # Query output for this TAK
    query = f"""
    SELECT ConceptName, StartDateTime, EndDateTime, Value, AbstractionType
    FROM OutputPatientData
    WHERE PatientId = {DEBUG_PATIENT_ID} AND ConceptName = '{tak_name}'
    ORDER BY StartDateTime
    """
    df_tak_output = pd.read_sql_query(query, debug_da.conn)
    
    print(f"{'='*80}")
    print(f"📌 {tak_name} ({tak.family})")
    print(f"   Derived from: {getattr(tak, 'derived_from', 'N/A')}")
    print(f"   Output rows: {len(df_tak_output)}")
    
    if not df_tak_output.empty:
        print(f"\n   Sample output (first 5 rows):")
        display(df_tak_output.head(5))
    else:
        print(f"\n   ⚠️  NO OUTPUT (check derived-from concepts above)")
    
    print()

In [ ]:
# ============================================================
# Step 5: Deep Dive - Manual TAK Execution with Debug Prints
# ============================================================

# Select a TAK to manually execute and inspect
INSPECT_TAK = "DIABETES_DIAGNOSIS_CONTEXT"

print(f"🔬 Deep Dive: {INSPECT_TAK}\n")
print("="*80)

# Get TAK
tak = debug_repo.get(INSPECT_TAK)
if not tak:
    print(f"❌ TAK '{INSPECT_TAK}' not found")
else:
    print(f"📋 TAK Info:")
    print(f"   Family: {tak.family}")
    print(f"   Derived from: {getattr(tak, 'derived_from', 'N/A')}")
    
    # Get input for this TAK (from cache or DB)
    if tak.family == "raw-concept":
        # Query InputPatientData directly
        attr_names = [attr['name'] for attr in tak.attributes]
        query = f"""
        SELECT PatientId, ConceptName, StartTime as StartDateTime, EndTime as EndDateTime, Value
        FROM InputPatientData
        WHERE PatientId = {DEBUG_PATIENT_ID} AND ConceptName IN ({','.join(f"'{a}'" for a in attr_names)})
        ORDER BY StartTime
        """
        df_input = pd.read_sql_query(query, debug_da.conn)
        df_input['AbstractionType'] = 'input'
    else:
        # Query OutputPatientData for derived-from TAK(s)
        if isinstance(tak.derived_from, list):
            # Event/Context/Pattern
            derived_names = [df['name'] for df in tak.derived_from]
        else:
            # State/Trend
            derived_names = [tak.derived_from]
        
        query = f"""
        SELECT PatientId, ConceptName, StartDateTime, EndDateTime, Value, AbstractionType
        FROM OutputPatientData
        WHERE PatientId = {DEBUG_PATIENT_ID} AND ConceptName IN ({','.join(f"'{n}'" for n in derived_names)})
        ORDER BY StartDateTime
        """
        df_input = pd.read_sql_query(query, debug_da.conn)
    
    print(f"\n📥 Input to {INSPECT_TAK}:")
    print(f"   Rows: {len(df_input)}")
    if not df_input.empty:
        print(f"   Concepts: {sorted(df_input['ConceptName'].unique())}")
        display(df_input.head(10))
    else:
        print(f"   ⚠️  NO INPUT FOUND")
        print(f"   Expected concepts: {derived_names if 'derived_names' in locals() else attr_names}")
    
    # Manually apply TAK
    if not df_input.empty:
        print(f"\n🔄 Applying {INSPECT_TAK}.apply()...\n")
        df_output = tak.apply(df_input)
        
        print(f"\n📤 Output from {INSPECT_TAK}:")
        print(f"   Rows: {len(df_output)}")
        if not df_output.empty:
            display(df_output.head(10))
        else:
            print(f"   ⚠️  NO OUTPUT (check abstraction rules / context windows)")

In [ ]:
# ============================================================
# Step 6: Cleanup Debug DB
# ============================================================

# Close connection
debug_da.conn.close()

# Optionally delete debug DB
# DEBUG_DB_PATH.unlink()
# print(f"🗑️  Deleted debug DB: {DEBUG_DB_PATH}")

print(f"\n✅ Debug session complete!")
print(f"   Debug DB preserved at: {DEBUG_DB_PATH}")
print(f"   (Delete manually or re-run this block to recreate)")